# TCC

In [38]:
import pandas as pd
import os as os
import PyPDF2
import fitz

import re

## Mapeando questões das provas

In [42]:
def map_questions(exam_string):    
    
    # Regex para o split usando "questão", espaço em branco e número (case-insensitive)
    pattern = re.compile(r'questão\s+\d+', re.IGNORECASE)
    result = pattern.split(exam_string)
    
    # Remover strings vazias do resultado
    return [item.strip() for item in result if item.strip()]

In [205]:
def separate_question(question):
        
    # Expressão regular para encontrar o corpo e as alternativas
    # pattern = re.compile(r'(\nA\s.*?\nB\s.*?\nC\s.*?\nD\s.*?\nE\s.*?\.)', question, re.DOTALL)
    # print(pattern)
    # print(pattern.findall(question))

    # Encontrar a correspondência na string
    alternatives_regex = r'\nA\s.*?\nB\s.*?\nC\s.*?\nD\s.*?\nE\s.*?\.'
    body = re.search(f'^(.*?)(?={alternatives_regex})', question, re.DOTALL).group(1)
    alternatives = re.search(f'({alternatives_regex})', question, re.DOTALL).group(1)    
    
    return ({'body': body, 'alternatives':  alternatives})


In [206]:
testando_regex = """O termo agronegócio não se refere apenas à agricultura 
e à pecuária, pois as atividades ligadas a essa produção 
incluem fornecedores de equipamentos, serviços para a 
zona rural, industrialização e comercialização dos produtos.
Os dados seguintes refletem a participação percentual 
do agronegócio no PIB brasileiro entre os anos de 1998 
e 2008:
• 
1998: 21,33%;
• 
1999: 22,24%;
• 
2000: 22,87%;
• 
2001: 23,26%;
• 
2002: 25,31%;
• 
2003: 28,28%;
• 
2004: 27,79%;
• 
2005: 25,83%;
• 
2006: 23,92%;
• 
2007: 24,74%;
• 
2008: 26,46%.
Esses dados foram citados em uma palestra na 
qual o orador ressaltou uma queda da participação do 
agronegócio no PIB brasileiro e a posterior recuperação 
dessa participação, em termos percentuais.
De acordo com essas informações, o período de queda 
ocorreu entre os anos de 
A 1998 e 2001.
B 2001 e 2003.
C 2003 e 2006.
D 2003 e 2007.
E 2003 e 2008."""

print(separate_question(testando_regex))

{'body': 'O termo agronegócio não se refere apenas à agricultura \ne à pecuária, pois as atividades ligadas a essa produção \nincluem fornecedores de equipamentos, serviços para a \nzona rural, industrialização e comercialização dos produtos.\nOs dados seguintes refletem a participação percentual \ndo agronegócio no PIB brasileiro entre os anos de 1998 \ne 2008:\n• \n1998: 21,33%;\n• \n1999: 22,24%;\n• \n2000: 22,87%;\n• \n2001: 23,26%;\n• \n2002: 25,31%;\n• \n2003: 28,28%;\n• \n2004: 27,79%;\n• \n2005: 25,83%;\n• \n2006: 23,92%;\n• \n2007: 24,74%;\n• \n2008: 26,46%.\nEsses dados foram citados em uma palestra na \nqual o orador ressaltou uma queda da participação do \nagronegócio no PIB brasileiro e a posterior recuperação \ndessa participação, em termos percentuais.\nDe acordo com essas informações, o período de queda \nocorreu entre os anos de ', 'alternatives': '\nA 1998 e 2001.\nB 2001 e 2003.\nC 2003 e 2006.\nD 2003 e 2007.\nE 2003 e 2008.'}


### Convertendo PDF em string

In [ ]:
 
doc = fitz.open('../TCC/pdfs/2011/2011_D2_CD6_cinza.pdf') 
text = "" 
splitter = ""
for page in doc: 
    if ("\nQuESTÃO" in page.get_text()):
        text+=page.get_text() 
    elif ("\nQUESTÃO" in page.get_text()):
        text+=page.get_text() 
    elif ("Questão" in page.get_text()):
        text+=page.get_text()     
        
teste = map_questions(text)

print(len(teste))
for i in range(len(teste)):
    print(teste[i])
    print("-------------------------------------------------")
    print("Questão separada")
    print("-------------------------------------------------")
    print(separate_question(teste[i]))
    print("-------------------------------------------------")
    print("-------------------------------------------------")

# string_teste = "Going to university seems to reduce the risk of dying from coronary heart disease. An American study that involved 10 000 patients from around the world has found that people who leave school before the age of 16 are five times more likely to suffer a heart attack and die than university graduates. Em relação às pesquisas, a utilização da expressão university graduates evidencia a intenção de informar que A as doenças do coração  atacam dez mil pacientes. B as doenças do coração ocorrem na faixa dos dezesseis anos. C as pesquisas sobre doenças são divulgadas no meio acadêmico. D jovens americanos são alertados dos riscos de doenças do coração. E maior nível de estudo reduz riscos de ataques do coração."
# print(separate_question(string_teste))

In [221]:
PATH = "../TCC/test_txt_files"
years = os.listdir(f'{PATH}')
for year in years:
    
    exams = os.listdir(f'{PATH}/{year}')
    all_questions_edition = ''
    for test in exams:

        text = "" 
        if( "_D2_" in test):
            text += "\n------D2------\n"
                
        doc = fitz.open(f'{PATH}/{year}/{test}') 
        
        for page in doc: 
            if ("\nQuESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("\nQUESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("Questão" in page.get_text()):
                text += page.get_text()         
        all_questions_edition += text
       
    with open(f'{PATH}/{year}/prova{year}.txt', 'w', encoding='utf-8') as exam_str:
        exam_str.write(all_questions_edition)
        
    break


### Lendo os arquivos com os textos das provas e mapeando as questões em um Dataframe

In [ ]:
def agg_question_info(question, year,number):
    
    if(int(year) > 2017):
        if(number <= 45):
            area = ''
        elif(number > 45 and number <= 90):
            area = ''
        elif(number > 90 and number <= 135):
            area = ''
        elif(number > 135 and number <= 180):
            area = ''
    else:
        if(number <= 45):
            area = ''
        elif(number > 45 and number <= 90):
            area = ''
        elif(number > 90 and number <= 135):
            area = ''
        elif(number > 135 and number <= 180):
            area = ''
        
    return pd.Series()

In [ ]:
enem_questions = pd.DataFrame(columns=['id','year','area','body','itens','result','answer'])

for year in years:
    
    test = os.listdir(f'{PATH}/{year}')

    with open(f'{PATH}/{year}/prova{year}.txt', 'r', encoding='utf-8') as exam_str:
        
        exam_string = exam_str.read()
        questions = map_questions(exam_string)
        counter = 0
        for question in questions:
            # separar corpo e alternativas
            counter += 1
            question_obj = separate_question(question)
            
            question_info = agg_question_info(question_obj, year, counter)
            
            enem_questions = enem_questions.append(question_info, ignore_index=True)
            # enem_questions = pd.DataFrame.concat([enem_questions, question_info], ignore_index=True)
    
    enem_questions.to_csv(f'{PATH}/csvs/enem_{year}_questions.csv', index=False)
    
            # print("-------------------------------------------------")
            # print("-------------------------------------------------")
            # print("-------------------------------------------------")
            
